In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [3]:
!pip install scikit-image

In [4]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from plot_utils import *
from image_utils import *
import mappings as mapps

In [5]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import re
from pyproj import Transformer
from datetime import date
from scipy.ndimage import label
import time
from tqdm import tqdm
import os

In [6]:
update_storage_access()

In [7]:
environment = get_environment()

root_path = get_root_path()

bucket = environment["bucket"]

#2022
#guyane 2022
path_s3_pleiades_data_2022_guyane = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_pleiades_data_2022_guyane = environment["local-path"]["PLEIADES"][2022]["guyane"]

#martinique 2022
path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]
          
#guadeloupe 2022
path_s3_pleiades_data_2022_guadeloupe = environment["sources"]["PLEIADES"][2022]["guadeloupe"]
path_local_pleiades_data_2022_guadeloupe = environment["local-path"]["PLEIADES"][2022]["guadeloupe"]

#mayotte 2022
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["mayotte"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["mayotte"]

#2020
#mayotte 2020
path_s3_pleiades_data_2020_mayotte = environment["sources"]["PLEIADES"][2020]["mayotte"]
path_local_pleiades_data_2020_mayotte = environment["local-path"]["PLEIADES"][2020]["mayotte"]

#2019
#mayotte 2019
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["mayotte"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["mayotte"]

#2018
#reunion 2018
path_s3_pleiades_data_2018_reunion = environment["sources"]["PLEIADES"][2018]["reunion"]
path_local_pleiades_data_2018_reunion = environment["local-path"]["PLEIADES"][2018]["reunion"]

#2017
#martinique 2017
path_s3_pleiades_data_2017_martinique = environment["sources"]["PLEIADES"][2017]["martinique"]
path_local_pleiades_data_2017_martinique = environment["local-path"]["PLEIADES"][2017]["martinique"]

#mayotte 2017
path_s3_pleiades_data_2017_mayotte = environment["sources"]["PLEIADES"][2017]["mayotte"]
path_local_pleiades_data_2017_mayotte = environment["local-path"]["PLEIADES"][2017]["mayotte"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [8]:
# DL PLEIADE mayotte 2020
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2020_mayotte}",
        lpath=f"../{path_local_pleiades_data_2020_mayotte}",
        recursive=True)

In [ ]:
# DL PLEIADE mayotte 2017
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_mayotte}",
        lpath=f"../{path_local_pleiades_data_2017_mayotte}",
        recursive=True)

## Détection nuages

In [37]:
plot_square_images([0,1,2], 2 , None, '../data/PLEIADES/2020/MAYOTTE/ORT_2020052526656219_0508_8599_U38S_8Bits.jp2')

Voici un exemple d'une image avec un nuage

In [13]:
#image avec nuage 
filename_3 = '../data/PLEIADES/2020/MAYOTTE/ORT_2020052526656219_0508_8599_U38S_8Bits.jp2' 
date_3 = date.fromisoformat('2020-01-01')

In [14]:
image_3 = SatelliteImage.from_raster(
        filename_3,
        date = date_3, 
        n_bands = 3,
        dep = "976"
    )
image_3.normalize()

In [15]:
image_3.plot([0,1,2]) 

In [19]:
import numpy as np
from scipy.ndimage import label
import matplotlib.pyplot as plt
import time

def count_patch_nuage(image):
    list_images = image.split(250)
    list_nuage = []
    num_patch = 0
    #on parcourt chaque patch
    for mini_image in list_images:
        image = mini_image.array.copy()

        image = image[[0,1,2],:, :]

        image = (image*255).astype(np.uint8)

        image = image.transpose(1, 2, 0)

        # Convertir l'image RGB en niveau de gris
        grayscale = np.mean(image, axis=2)

        # Trouver les amas de pixels blancs de taille 10 ou plus
        threshold = 250  # seuil pour considérer un pixel comme "blanc"

        labeled, num_features = label(grayscale > threshold)

        nuage=False
        min_size = 20000 # taille minimale de l'amas
        if num_features >= 1:
            lab = np.asarray(labeled)
            for i in range(1, num_features + 1):
                if np.sum(lab == i) >= min_size: 
                    nuage= True
                    break
        
        if nuage == True:
            list_nuage.append(num_patch)
        
        num_patch +=1
            
    return(list_nuage)

def mask_nuage(image, threshold, min_size):
    image = image.array.copy()

    image = image[[0,1,2],:, :]

    image = (image*255).astype(np.uint8)

    image = image.transpose(1, 2, 0)

    # Convertir l'image RGB en niveau de gris
    grayscale = np.mean(image, axis=2)

    # Trouver les amas de pixels blancs correspondant à 5% ou plus de l'image
    #threshold = 250  # seuil pour considérer un pixel comme "blanc"

    labeled, num_features = label(grayscale > threshold)

    #min_size = 20000  # taille minimale de l'amas (2000*2000 = 4 000 000 pixels et on souhaite détecter des nuages qui occupent + de 0,5% de l'image)
    mask = np.zeros_like(grayscale, dtype=bool)
    if num_features >= 1:
        lab = np.asarray(labeled)
        for i in tqdm(range(1, num_features + 1)): # Affichage de la barre de progression
            if np.sum(lab == i) >= min_size: 
                mask |= np.array(labeled == i)

    # Afficher l'image avec les amas de pixels blancs en rouge
    result = np.copy(image)
    result[mask] = [255, 0, 0]
    
    return(result)
            
def plot_nuage(image, threshold, min_size):
    image = image.array.copy()

    image = image[[0,1,2],:, :]

    image = (image*255).astype(np.uint8)

    image = image.transpose(1, 2, 0)

    # Convertir l'image RGB en niveau de gris
    grayscale = np.mean(image, axis=2)

    # Trouver les amas de pixels blancs correspondant à 5% ou plus de l'image
    #threshold = 250  # seuil pour considérer un pixel comme "blanc"

    labeled, num_features = label(grayscale > threshold)

    #min_size = 20000  # taille minimale de l'amas (2000*2000 = 4 000 000 pixels et on souhaite détecter des nuages qui occupent + de 0,5% de l'image)
    mask = np.zeros_like(grayscale, dtype=bool)
    if num_features >= 1:
        lab = np.asarray(labeled)
        for i in tqdm(range(1, num_features + 1)): # Affichage de la barre de progression
            if np.sum(lab == i) >= min_size: 
                mask |= np.array(labeled == i)

    # Afficher l'image avec les amas de pixels blancs en rouge
    result = np.copy(image)
    result[mask] = [255, 0, 0]

    # Afficher l'image originale et l'image résultante côte à côte
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image)
    ax1.set_title("Image originale")
    ax2.imshow(result)
    ax2.set_title("Image avec les amas de pixels blancs en rouge")
    plt.show()
    
def detect_nuage_folder(folder_path, threshold, min_size):
    nuages = []
    filenames = os.listdir(folder_path)[:100]
    
    for filename in tqdm(filenames): # Affichage de la barre de progression
        try :
        
            image = SatelliteImage.from_raster(
                    folder_path + '/' +filename,
                    date = date.fromisoformat('2020-01-01'), 
                    n_bands = 3,
                    dep = "976"
                    )
            image.normalize()

            list_images = image.split(250)

            #on parcourt chaque patch
            for mini_image in list_images:

                image = mini_image.array.copy()

                image = image[[0,1,2],:, :]

                image = (image*255).astype(np.uint8)

                image = image.transpose(1, 2, 0)

                # Convertir l'image RGB en niveau de gris
                grayscale = np.mean(image, axis=2)

                # Trouver les amas de pixels blancs de taille 10 ou plus
                #threshold = 250  # seuil pour considérer un pixel comme "blanc"

                labeled, num_features = label(grayscale > threshold)

                nuage=False
                #min_size = 20000 # taille minimale de l'amas (on souhaite détecter des nuages qui occupent + de 0,5% de l'image TOTALE !)
                if num_features >= 1:
                    lab = np.asarray(labeled)
                    for i in range(1, num_features + 1):
                        if np.sum(lab == i) >= min_size:
                            nuage= True
                            break
                        else:
                            continue

                if nuage == True:
                    nuages.append(folder_path + '/' +filename)
                    break
                    
                else :
                    continue

        except RasterioIOError:
            print("L'image "+filename+" ne s'ouvre pas.")
            
        finally :
            continue
            
    return(nuages)



def detect_nuage_list(list_path, threshold, min_size):
    nuages = []
    
    for filename in tqdm(list_path): # Affichage de la barre de progression
        try :
        
            image = SatelliteImage.from_raster(
                    filename,
                    date = date.fromisoformat('2020-01-01'), 
                    n_bands = 3,
                    dep = "976"
                    )
            image.normalize()

            image = image.array.copy()

            image = image[[0,1,2],:, :]

            image = (image*255).astype(np.uint8)

            image = image.transpose(1, 2, 0)

            # Convertir l'image RGB en niveau de gris
            grayscale = np.mean(image, axis=2)

            # Trouver les amas de pixels blancs de taille 10 ou plus
            threshold = 250  # seuil pour considérer un pixel comme "blanc"

            labeled, num_features = label(grayscale > threshold)

            nuage=False
            min_size = 20000 # taille minimale de l'amas (on souhaite détecter des nuages qui occupent + de 0,5% de l'image TOTALE !)
            if num_features >= 1:
                lab = np.asarray(labeled)
                for i in range(1, num_features + 1):
                    if np.sum(lab == i) >= min_size:
                        # cpt +=1
                        nuage= True
                        break
                    else:
                        continue

            if nuage == True:
                nuages.append(folder_path + '/' +filename)
                break

            else :
                continue

        except RasterioIOError:
            print("L'image "+filename+" ne s'ouvre pas.")
            
        finally :
            continue
            
    return(nuages)


def mask_nuage_filepaths(list_path, threshold, min_size):
    masks = []
    
    for filename in tqdm(list_path): # Affichage de la barre de progression
        try :
        
            image = SatelliteImage.from_raster(
                    filename,
                    date = date.fromisoformat('2020-01-01'), 
                    n_bands = 3,
                    dep = "976"
                    )
            image.normalize()
            
            image = image.array.copy()

            image = image[[0,1,2],:, :]

            image = (image*255).astype(np.uint8)

            image = image.transpose(1, 2, 0)

            # Convertir l'image RGB en niveau de gris
            grayscale = np.mean(image, axis=2)

            # Trouver les amas de pixels blancs correspondant à 5% ou plus de l'image
            #threshold = 250  # seuil pour considérer un pixel comme "blanc"

            labeled, num_features = label(grayscale > threshold)

            #min_size = 20000  # taille minimale de l'amas (2000*2000 = 4 000 000 pixels et on souhaite détecter des nuages qui occupent + de 0,5% de l'image)
            mask = np.zeros_like(grayscale, dtype=bool)
            if num_features >= 1:
                lab = np.asarray(labeled)
                for i in range(1, num_features + 1):
                    if np.sum(lab == i) >= min_size: 
                        mask |= np.array(labeled == i)

            # Afficher l'image avec les amas de pixels blancs en rouge
            result = np.copy(image)
            result[mask] = [255, 0, 0]
            masks.append(result)
        
        except RasterioIOError:
            print("L'image "+filename+" ne s'ouvre pas.")
            
        finally :
            continue
            
    return(masks)

In [125]:
nuages = detect_nuage_folder('../data/PLEIADES/2020/MAYOTTE', threshold = 250, min_size = 20000)

In [126]:
len(nuages)

In [127]:
nuages2 = detect_nuage_folder('../data/PLEIADES/2020/MAYOTTE', threshold = 160, min_size = 30000)

In [128]:
len(nuages2)

In [226]:
nuages3 = detect_nuage_folder('../data/PLEIADES/2020/MAYOTTE', threshold = 240, min_size = 30000)

In [225]:
len(nuages3)

In [223]:
image_test = SatelliteImage.from_raster(
        nuages[10],
        date = date.fromisoformat('2020-01-01'), 
        n_bands = 3,
        dep = "976"
        )
image_test.normalize()
image_test.plot([0,1,2])
#plot_nuage(image_test)

In [42]:
def classify_clusters(image, threshold, min_size):
    image = image.array.copy()

    image = image[[0,1,2],:, :]

    image = (image*255).astype(np.uint8)

    image = image.transpose(1, 2, 0)

    # Convertir l'image RGB en niveau de gris
    grayscale = np.mean(image, axis=2)

    # Label the connected components in the binary image
    labeled, num_features = label(grayscale > threshold)

    # Initialize an array to store the classification results
    result = np.zeros_like(grayscale, dtype=np.uint8)

    # Classify each connected component based on its size
    for i in tqdm(range(1, num_features+1)):
        # Calculate the size of the current component
        size = np.sum(labeled == i)

        # If the component is larger than the minimum size, classify it as a cluster
        if size >= min_size:
            result[labeled == i] = 255
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image)
    ax1.set_title("Image originale")
    ax2.imshow(result)
    ax2.set_title("Image avec les amas de pixels blancs en rouge")
    plt.show()
    



In [150]:
result = classify_clusters(image_test, 200, 90000)

In [154]:
os.listdir('../data/PLEIADES/2020/MAYOTTE')[:100][24]

In [170]:
import re

In [205]:
folder_path = '../data/PLEIADES/2020/MAYOTTE'
distance = 2
bands_indices = [0,1,2]

# Get folder path
pattern = "/"

split_folder_path = re.split(pattern, folder_path)


# Retrieve the year and the department
annee = split_folder_path[3]
departement = split_folder_path[4]
dep_num = name_dep_to_num_dep[departement]


list_images = []

for filename in os.listdir('../data/PLEIADES/2020/MAYOTTE')[75:100]:

# Retrieve left-top coordinates of all images
        image = SatelliteImage.from_raster(
                folder_path + '/' +filename,
                date = date.fromisoformat(annee + '-01-01'), 
                n_bands = len(bands_indices),
                dep = dep_num
            )
        image.normalize()
        list_images.append(image)

mat_list_images = np.transpose(np.array(list_images).reshape(5,5))


# Create a figure and axes
fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(10,10))

# Iterate over the grid of masks and plot them
for i in range(5):
    for j in range(5):
        axs[i, j].imshow(
            mat_list_images[i, j].array.transpose(2,1,0)
        )

# Remove any unused axes
for i in range(5):
    for j in range(5):
        axs[i, j].set_axis_off()

# Show the plot
plt.show()
        

In [216]:
import math

In [219]:
def plot_list_path_square(list_filepaths):
    size = int(math.sqrt(len(list_filepaths)))
    folder_path = list_filepaths[0]
    bands_indices = [0,1,2]

    # Get folder path
    pattern = "/"

    split_folder_path = re.split(pattern, folder_path)


    # Retrieve the year and the department
    annee = split_folder_path[3]
    departement = split_folder_path[4]
    dep_num = name_dep_to_num_dep[departement]


    list_images = []

    for filepath in list_filepaths:

    # Retrieve left-top coordinates of all images
            image = SatelliteImage.from_raster(
                    filepath,
                    date = date.fromisoformat(annee + '-01-01'), 
                    n_bands = len(bands_indices),
                    dep = dep_num
                )
            image.normalize()
            list_images.append(image)

    mat_list_images = np.transpose(np.array(list_images).reshape(size,size))


    # Create a figure and axes
    fig, axs = plt.subplots(nrows=size, ncols=size, figsize=(10,10))

    # Iterate over the grid of masks and plot them
    for i in range(size):
        for j in range(size):
            axs[i, j].imshow(
                mat_list_images[i, j].array.transpose(2,1,0)
            )

    # Remove any unused axes
    for i in range(size):
        for j in range(size):
            axs[i, j].set_axis_off()

    # Show the plot
    plt.show()


In [227]:
plot_list_path_square(nuages[:9])

In [ ]:
masks=mask_nuage_filepaths(nuages, 160, 20000)

In [17]:
import geopandas as gpd
from shapely.geometry import Polygon
from rasterio.features import rasterize, shapes

In [49]:
nuage_centre = mask_nuage(image_3, threshold = 250, min_size = 20000)
nuage_entier = mask_nuage(image_3, threshold = 160, min_size = 20000)

In [55]:
image_height = image_3.array.shape[1]
image_width = image_3.array.shape[2]
polygon_list = []
for shape in list(shapes(nuage_centre)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * image_height * image_width:
        continue
    polygon_list.append(polygon)
    
# Créer un GeoSeries à partir de la liste de polygons
g_centre = gpd.GeoDataFrame(geometry = polygon_list)

In [57]:
polygon_list = []
for shape in list(shapes(nuage_entier)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * image_height * image_width:
        continue
    polygon_list.append(polygon)
 
#Créer un GeoSeries à partir de la liste de polygons
g_entier = gpd.GeoDataFrame(geometry = polygon_list)

In [59]:
result = gpd.sjoin(g_entier, g_centre,how="inner",predicate = "intersects")

In [60]:
result.geometry

In [61]:
g_result = result.geometry

In [64]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_3.array, (1, 2, 0))[:,:,:3])
result.plot(color = "orange", ax=ax)